# MCBU CH4 flight planning tools

In [1]:
import os
import datetime
import json
import shutil

import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import arcgis
from arcgis.gis import GIS
from arcgis.features import GeoAccessor
import contextily as cx

# Flight planning polygons

## Connect to MapHub

In [33]:
gis = GIS("https://maphub-amer.tst.azure.chevron.com/portal", client_id='bb6vn14P957pN6bV')
print("Logged in as: " + gis.properties.user.username)

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://maphub-amer.tst.azure.chevron.com/portal/sharing/rest//oauth2/authorize?response_type=code&client_id=bb6vn14P957pN6bV&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=qakX47XJ0IJPaohHNnhPDCUD5jZOxO
Enter code obtained on signing in using SAML: ········


C:\Users\ekqo\Miniconda3\envs\arcgis\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maphub-amer.tst.azure.chevron.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Logged in as: ekqo


## Get wells data

In [40]:
# Title for wells data
title = 'COMM_MCBU_SOR_EAM_FACILITY'

# Search for the wells data by title
wells_id = gis.content.search(query='title:' + title,
                               item_type='Feature Layer',
                               max_items=30)

# Get the wells data feature layer
wells_data = gis.content.get('857b2093bff44132b39e65176c309284').layers[6]

# .query() returns a FeatureSet
fset = wells_data.query()

# get a GeoJSON string representation of the FeatureSet
gjson_string = fset.to_geojson

# read GeoJSON string into a dict
gjson_dict = json.loads(gjson_string)

# Read the dict into a gdf
wells_gdf = gpd.GeoDataFrame.from_features(gjson_dict['features'])

# Set the CRS
wells_gdf = wells_gdf.set_crs(epsg=4267)
wells_gdf

,geometry,OBJECTID,FOT_ASSET_NAME,LOC_AREA,PARENT_OBJ_CODE,PARENT_OBJ_CLASS,PARENT_OBJ_DESC,CHILD_OBJ_CODE,CHILD_OBJ_CLASS,CHILD_OBJ_DESC,...,TANK_COLOR,BTU_RATE,HP,TANK_ENG_SERVICE,MCBU_GIS_DATA_UPLOADED_BY,MCBU_GIS_DATA_SOURCE_NAME,MCBU_GIS_DATA_SOURCE_TYPE,MCBU_GIS_DATA_REMARKS,MCBU_GIS_DATA_RECEIVED_DATE,MCBU_GIS_DATA_UPLOADED_DATE
0,POINT (-103.51996 32.79684),37330674,Buckeye / Vacuum,CENTRAL,BCT49CVUBP2HDR,FLNHDR,HEADER; CVU P2,FA3393,OSSWL,WELL; CVU 54 VGSA,...,None,None,None,None,svc-mcbu-gis-fmeT,FDS DATABASE,MS_SQL_VIEW,,1647993600000,1647993600000
1,POINT (-103.51972 32.78799),37330680,Buckeye / Vacuum,CENTRAL,BCT49CVUIJI3HDR,FLNHDR,HEADER; CVU I3,NW2155,OSSWL,WELL; CVU 181 VGSA,...,None,None,None,None,svc-mcbu-gis-fmeT,FDS DATABASE,MS_SQL_VIEW,,1647993600000,1647993600000
2,POINT (-103.49888 32.78761),37330681,Buckeye / Vacuum,CENTRAL,BCT49CVUIJI4HDR,FLNHDR,HEADER; CVU I4,ON7423,OSSWL,WELL; CVU 345 VGSA,...,None,None,None,None,svc-mcbu-gis-fmeT,FDS DATABASE,MS_SQL_VIEW,,1647993600000,1647993600000
3,POINT (-103.49714 32.77057),37330682,Buckeye / Vacuum,CENTRAL,BCT49CVUIJI6HDR,FLNHDR,HEADER; CVU I6,EQ3040,OSSWL,WELL; CVU 121 VGSA,...,None,None,None,None,svc-mcbu-gis-fmeT,FDS DATABASE,MS_SQL_VIEW,,1647993600000,1647993600000
4,POINT (-103.50644 32.78606),37330688,Buckeye / Vacuum,CENTRAL,BCT49CVUBP3HDR,FLNHDR,HEADER; CVU P3,BI4106,OSSWL,WELL; CVU 188 VGSA,...,None,None,None,None,svc-mcbu-gis-fmeT,FDS DATABASE,MS_SQL_VIEW,,1647993600000,1647993600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5868,POINT (-102.27396 31.73953),37404032,Odessa West,MIDLAND,PA8LCBBY,OSEBTY,BATTERY; LUPIN CENTRAL TANK BTY,LZ4498,OSSWL,WELL; LUPIN TXL FEE 3101M,...,None,None,None,None,svc-mcbu-gis-fmeT,FDS DATABASE,MS_SQL_VIEW,,1647993600000,1647993600000
5869,POINT (-102.29058 31.70941),37404035,Odessa West,MIDLAND,PA8LSTXLFEE37SB,OSEBTY,BATTERY; LUPIN TXL FEE SEC 37 SAT,NC5541,OSSWL,WELL; LUPIN TXL FEE 3715ER,...,None,None,None,None,svc-mcbu-gis-fmeT,FDS DATABASE,MS_SQL_VIEW,,1647993600000,1647993600000
5870,POINT (-102.35241 31.70309),37404040,Odessa West,MIDLAND,PA8LWEBY,OSEBTY,BATTERY; LUPIN WEST EDGE CENTRAL BTY,NN7903,OSSWL,WELL; LUPIN WE TXL FEE B 4113E,...,None,None,None,None,svc-mcbu-gis-fmeT,FDS DATABASE,MS_SQL_VIEW,,1647993600000,1647993600000
5871,POINT (-102.17613 31.89571),37404045,Odessa West,MIDLAND,PF8BGSEC28SB,OSEBTY,BATTERY; BRYANT G SEC 28 SAT,PC0968,OSSWL,WELL; C SCHARBAUER A 2805M,...,None,None,None,None,svc-mcbu-gis-fmeT,FDS DATABASE,MS_SQL_VIEW,,1647993600000,1647993600000


## Create buffers around points

In [45]:
# Create copy of the wells point data
wells_poly = wells_gdf.copy()

# Reproject to UTM Zone 14N
wells_poly = wells_poly.to_crs("EPSG:32614")

# Buffer each point using a 20 meter circle radius
# and replace the point geometry with the new buffered geometry
wells_poly["geometry"] = wells_poly.geometry.buffer(100)
wells_poly.head()

,geometry,OBJECTID,FOT_ASSET_NAME,LOC_AREA,PARENT_OBJ_CODE,PARENT_OBJ_CLASS,PARENT_OBJ_DESC,CHILD_OBJ_CODE,CHILD_OBJ_CLASS,CHILD_OBJ_DESC,...,TANK_COLOR,BTU_RATE,HP,TANK_ENG_SERVICE,MCBU_GIS_DATA_UPLOADED_BY,MCBU_GIS_DATA_SOURCE_NAME,MCBU_GIS_DATA_SOURCE_TYPE,MCBU_GIS_DATA_REMARKS,MCBU_GIS_DATA_RECEIVED_DATE,MCBU_GIS_DATA_UPLOADED_DATE
0,"POLYGON ((76670.265 3637837.444, 76669.783 363...",37330674,Buckeye / Vacuum,CENTRAL,BCT49CVUBP2HDR,FLNHDR,HEADER; CVU P2,FA3393,OSSWL,WELL; CVU 54 VGSA,...,None,None,None,None,svc-mcbu-gis-fmeT,FDS DATABASE,MS_SQL_VIEW,,1647993600000,1647993600000
1,"POLYGON ((76650.733 3636854.140, 76650.252 363...",37330680,Buckeye / Vacuum,CENTRAL,BCT49CVUIJI3HDR,FLNHDR,HEADER; CVU I3,NW2155,OSSWL,WELL; CVU 181 VGSA,...,None,None,None,None,svc-mcbu-gis-fmeT,FDS DATABASE,MS_SQL_VIEW,,1647993600000,1647993600000
2,"POLYGON ((78602.946 3636728.524, 78602.464 363...",37330681,Buckeye / Vacuum,CENTRAL,BCT49CVUIJI4HDR,FLNHDR,HEADER; CVU I4,ON7423,OSSWL,WELL; CVU 345 VGSA,...,None,None,None,None,svc-mcbu-gis-fmeT,FDS DATABASE,MS_SQL_VIEW,,1647993600000,1647993600000
3,"POLYGON ((78685.600 3634830.173, 78685.118 363...",37330682,Buckeye / Vacuum,CENTRAL,BCT49CVUIJI6HDR,FLNHDR,HEADER; CVU I6,EQ3040,OSSWL,WELL; CVU 121 VGSA,...,None,None,None,None,svc-mcbu-gis-fmeT,FDS DATABASE,MS_SQL_VIEW,,1647993600000,1647993600000
4,"POLYGON ((77886.848 3636586.477, 77886.366 363...",37330688,Buckeye / Vacuum,CENTRAL,BCT49CVUBP3HDR,FLNHDR,HEADER; CVU P3,BI4106,OSSWL,WELL; CVU 188 VGSA,...,None,None,None,None,svc-mcbu-gis-fmeT,FDS DATABASE,MS_SQL_VIEW,,1647993600000,1647993600000


In [46]:
# Export the wells buffers as .shp
wells_poly.to_file('wells_buffered.shp')

C:\Users\ekqo\Miniconda3\envs\arcgis\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
C:\Users\ekqo\AppData\Local\Temp\ipykernel_27792\2834015157.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  wells_poly.to_file('wells_buffered.shp')
